In [ ]:
import aoc
import numpy as np
import math
import re
import itertools
from functools import reduce
import networkx as nx
import matplotlib.pyplot as plt

# Day 3

In [ ]:
raw_data = aoc.get_data(2020, 3)

In [ ]:
data = raw_data.replace('.', '0').replace('#', '1').splitlines()

In [ ]:
data

In [ ]:
patch = np.array([[int(y) for y in x] for x in data])

In [ ]:
slope = (3, 1)

In [ ]:
def make_forest(patch, slope):
    
    shape = patch.shape
    
    max_down = shape[0] * slope[1]
    
    max_right = max_down * slope[0]
    
    return np.concatenate([patch] * math.ceil(max_right / shape[1]), axis=1)

In [ ]:
forest = make_forest(patch, (1, 3))

In [ ]:
forest.shape

In [ ]:
def count_trees(forest, slope):
    
    x = 0
    y = 0
    
    trees = []
    
    while True:
        try: 
            x += slope[0]
            y += slope[1]
        
            trees.append(forest[y, x])
        except IndexError:
            break
    
    return sum(trees)

In [ ]:
count_trees(forest, slope)

# Day 4

In [ ]:
raw_data = aoc.get_data(2020, 4)

In [ ]:
passports = raw_data.split('\n\n')

In [ ]:
test = """eyr:1972 cid:100
hcl:#18171d ecl:amb hgt:170 pid:186cm iyr:2018 byr:1926

iyr:2019
hcl:#602927 eyr:1967 hgt:170cm
ecl:grn pid:012533040 byr:1946

hcl:dab227 iyr:2012
ecl:brn hgt:182cm pid:021572410 eyr:2020 byr:1992 cid:277

hgt:59cm ecl:zzz
eyr:2038 hcl:74454a iyr:2023
pid:3556412378 byr:2007""".split('\n\n')

In [ ]:
test

In [ ]:
required_keys = {'byr', 'iyr', 
'eyr' ,
'hgt' ,
'hcl' ,'ecl', 
'pid' ,
}

In [ ]:
required_keys

In [ ]:
def validate_all_passport_fields(passport):
    
    items = passport.replace(' ', '\n').splitlines()
    
    document = dict()
    
    for item in items:
    
        key, value = item.split(':')
        
        document[key] = value
    
    if set(document.keys()).intersection(required_keys) == required_keys:
        
        return document
    
    
def valid_between(value, low, high):
    
    value = int(value)
    
    return low <= value <= high


def valid_height(value):

    if re.search('\d*in', value) is not None:
        
        return valid_between(value[:-2], 59, 76)
    
    elif re.search('\d*cm', value) is not None:
        
        return valid_between(value[:-2], 150, 193)
        
    else:
        return False

def valid_hair(value):

    return re.search('#[0-9a-f]{6}', value) is not None

def valid_eye(value):

    return value in {'amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth'}

def valid_passport_number(value):
    
    return re.search('^\d{9}$', value) is not None

    
def validate_passport(passport):
    
    document = validate_all_passport_fields(passport)
    
    if document:
        
        return all([
            valid_between(document['byr'], 1920, 2002),
            valid_between(document['iyr'], 2010, 2020),
            valid_between(document['eyr'], 2020, 2030),
            valid_height(document['hgt']),
            valid_hair(document['hcl']), 
            valid_eye(document['ecl']), 
            valid_passport_number(document['pid'])])
    else:
        return False
            
        

In [ ]:
sum(map(lambda x: x is not None , map(validate_all_passport_fields, passports)))

In [ ]:
sum(map(validate_passport, passports))

# Day 5

In [ ]:
raw_data = aoc.get_data(2020, 5)

In [ ]:
data = raw_data.splitlines()

In [ ]:
def search(string, array, low):
    
    array_length = len(array)
    
    if array_length == 1:
        return array[0]
    
    comparitor = [*string].pop(0)
    
    if comparitor == low:
        
        return search(string[1:], array[: array_length//2], low)
    else:
        
        return search(string[1:], array[array_length//2:], low)

In [ ]:
def get_seat(code):
    
    row_code = code[:-3]
    col_code = code[-3:]
    
    row = search(row_code, list(range(128)), 'F')
    col = search(col_code, list(range(8)), 'L')
    
    return row * 8 + col
    

In [ ]:
seats = set(map(get_seat, data))

In [ ]:
max(seats)

In [ ]:
all_seats = set(range(min(seats), max(seats)))

In [ ]:
all_seats.difference(seats)

# Day 6 

In [ ]:
raw_data = aoc.get_data(2020, 6)

In [ ]:
data = raw_data.split('\n\n')

In [ ]:
def count_unique_questions(group):
    
    return len(set(group.replace('\n', '')))

In [ ]:
sum(map(count_unique_questions, data))

In [ ]:
def count_common_questions(group):
    
    people = [set(x) for x in group.splitlines()]
    
    return len(reduce(lambda a,b: a.intersection(b) , people))

In [ ]:
sum(map(count_common_questions, data))

# Day 7

In [ ]:
raw_data = aoc.get_data(2020,7)

In [ ]:
data = raw_data.splitlines()

In [ ]:
def make_bag_dict(bag_rules):
    
    bags = dict()
    
    for rule in bag_rules:
        
        color, contents = rule.replace('.', '').split(' bags contain ')
    
        contents = contents.split(', ')
        
        parsed_contents = list()
        
        for content in contents:
        
            content = re.search('\d+\s(.*)\sbags?', content)
            
            if content is None:
                
                bags[color] = None
            
            else:
                parsed_contents.append(content.group(1))
        
        bags[color] = parsed_contents
        
    return bags


In [ ]:
test = """light red bags contain 1 bright white bag, 2 muted yellow bags.
dark orange bags contain 3 bright white bags, 4 muted yellow bags.
bright white bags contain 1 shiny gold bag.
muted yellow bags contain 2 shiny gold bags, 9 faded blue bags.
shiny gold bags contain 1 dark olive bag, 2 vibrant plum bags.
dark olive bags contain 3 faded blue bags, 4 dotted black bags.
vibrant plum bags contain 5 faded blue bags, 6 dotted black bags.
faded blue bags contain no other bags.
dotted black bags contain no other bags."""

In [ ]:
rules = make_bag_dict(data)
len(rules.keys())

In [ ]:
target = 'shiny gold'

In [ ]:
rules

In [ ]:
def find_target_bag(bag, target):
    
    contents = rules[bag]
    
    if type(contents) == bool:
        return contents
    
    if target in contents:
        
        rules[bag] = True
        
        return True
    else:
        rules[bag] = any([find_target_bag(x, target) for x in contents])
    

In [ ]:
for bag in rules.keys():
    find_target_bag(bag, target)

In [ ]:
sum(rules.values())

In [ ]:
def construct_rule_network(rules):
    
    graph = nx.DiGraph()
    
    for rule in rules:
        
        color, contents = rule.replace('.', '').split(' bags contain ')
        
        color = re.findall('^(.*?)\sbags?', rule)
        
        contents = re.findall('\d+\s(.*?)\sbags?', rule)
    
        if contents:
            
            graph.add_edges_from(zip(color*len(contents), contents))
        
        
    return graph
    

In [ ]:
graph = construct_rule_network(data)

In [ ]:
pos = nx.nx_agraph.pygraphviz_layout(graph, prog='dot')

fig, ax = plt.subplots(figsize=(16,9))
nx.draw_networkx(graph, pos, ax=ax)

In [ ]:
sum(map(lambda x: nx.has_path(graph, x, target) if x != target else False, list(graph.nodes)))

In [ ]:
def construct_weighted_rule_network(rules):
    
    g = nx.DiGraph()
    
    for rule in rules:
        
        color, contents = rule.replace('.', '').split(' bags contain ')
        
        color = re.findall('^(.*?)\sbags?', rule)
        
        contents = re.findall('\d+\s(.*?)\sbags?', rule)
        
        number = map(int, re.findall('(\d+)\s.*?\sbags?', rule))
    
        if contents:
            
            g.add_weighted_edges_from(zip(color*len(contents), contents, number))
        
        
    return g

In [ ]:
graph = construct_weighted_rule_network(data)

In [ ]:
decendents = nx.descendants(graph, target)

decendents.add(target)

sub_graph = graph.subgraph(decendents)

In [ ]:
pos = nx.nx_agraph.pygraphviz_layout(sub_graph, prog='dot')

fig, ax = plt.subplots(figsize=(16,9))
nx.draw_networkx(sub_graph, pos, ax=ax)

In [ ]:
def count_bag_contents(start):
    
    
    decendants = nx.descendants_at_distance(sub_graph, start, 1)
    
    if not decendants:
        return 1
    
    return sum(count_bag_contents(decendant) * sub_graph[start][decendant]['weight'] for decendant in decendants) + 1
        
    
    
    

In [ ]:
count_bag_contents('shiny gold') - 1

# Day 8

In [ ]:
raw_data = aoc.get_data(2020,8)

In [ ]:
data = raw_data.splitlines()

In [ ]:
class Console:
    
    
    def __init__(self):
        
        self.accumulator = 0
        self.pointer = 0
        self.instructions = set()
        self.data = None
        
        
    def run(self, data):
    
        self.accumulator = 0
        self.pointer = 0
        self.instructions = set()
        self.data = data
        
        while self.pointer not in self.instructions:
            
            self.next_instruction()
            
        return self.accumulator
    
    
    
    def next_instruction(self):
        
        self.instructions.add(self.pointer)
        
        instruction, value = self.data[self.pointer].split(' ')
        
        match instruction:
            
            case 'nop':
                
                self.pointer += 1
                
            case 'acc':
                self.accumulator += int(value)
                self.pointer += 1
                
            case 'jmp':
                self.pointer += int(value)
        

In [ ]:
console = Console()

In [ ]:
console.run(data)

In [ ]:
def make_new_tape(tape):
    
    for i in range(len(tape)):
                       
        verb = tape[i]
        new_tape = tape.copy()
        
        
        if 'nop' in verb:
        
            new_tape[i] = verb.replace('nop', 'jmp')
            
            
            
        elif 'jmp' in verb:
            
            new_tape[i] = verb.replace('jmp', 'nop')
            
                      
        yield new_tape.copy()
    

In [ ]:
a = make_new_tape(data)

In [ ]:
for tape in make_new_tape(data):
    
    try:
        console.run(tape)
    except IndexError:
        print(console.accumulator)

# Day 9

In [ ]:
raw_data = aoc.get_data(2020,9)

In [ ]:
data = list(map(int, raw_data.splitlines()))

In [ ]:
list(map(sum, itertools.permutations(data[0:25],2)))

In [ ]:
def find_invalid_number(data):
    
    preamble = data[0:25]
    
    check_values = data[25:]
    
    
    for value in check_values:
        
        if value not in list(map(sum, itertools.permutations(preamble,2))):
        
            return value
        
        preamble.pop(0)
        preamble.append(value)
        

In [ ]:
find_invalid_number(data)

In [ ]:
def find_invalid_number_with_preamble(data):
    
    preamble = data[0:25]
    
    check_values = data[25:]
    
    
    for value in check_values:
        
        if value not in list(map(sum, itertools.permutations(preamble,2))):
        
            return value, preamble
        
        preamble.pop(0)
        preamble.append(value)

In [ ]:
value, preamble = find_invalid_number_with_preamble(data)

In [ ]:
def sliding_window(data, window_size):
    
    
    for i in range(len(data) - window_size + 1):
        
        yield data[i:i+window_size]

In [ ]:
list(sliding_window(preamble, 3))

In [ ]:
def find_values_in_preamble(value, preamble):
    
    for window_size in range(3, len(preamble)+1):
                        
        for window in sliding_window(preamble, window_size):
            
            print(sum(window))
            
            if sum(window) == value:
                
                return min(window) + max(window)
    

In [ ]:
find_values_in_preamble(value, preamble)

# Day 10

In [ ]:
raw_data = aoc.get_data(2020, 10)

In [ ]:
data = list(map(int, raw_data.splitlines()))

In [ ]:
max(data)

In [ ]:
adaptor_joltage = max(data) + 3

In [ ]:
joltages = [0] + sorted(data) + [adaptor_joltage]

In [ ]:
joltages

In [ ]:
def diff_1(value):
    
    return value[1] - value[0] == 1


def diff_3(value):
    
    return value[1] - value[0] == 3


In [ ]:
sum(map(diff_1, aoc.sliding_window(joltages, 2))) * sum(map(diff_3, aoc.sliding_window(joltages, 2)))

In [ ]:
sum(map(diff_1, aoc.sliding_window(joltages, 2)))

# Day 11

In [ ]:
raw_data = aoc.get_data(2020, 11)

In [ ]:
raw_data.splitlines()

In [ ]:
data = np.array([[y for y in x ] for x in raw_data.splitlines()])

In [ ]:
data

In [ ]:
def fill_seat_rule(seat):
    
    # seat will be filled if it is empty already and no neigbhors
    
    return '#' not in seat and seat[1,1] != '.'

In [ ]:
def vacate_seat_rule(seat):

    filled = seat[1,1] == '#'
    
    surrounding = (seat == "#").sum() > 4

    return filled and surrounding
    
    

In [ ]:
def get_seat(seats, row, col):
    
    return seats[row - 1: row+2, col-1:col+2]

In [ ]:
def next_turn(seats_grid):
    
    pad = np.pad(seats_grid, 1, constant_values='.')
    
    update = seats_grid.copy()
    
    for row in range(seats.shape[0]):
        
        for col in range(seats.shape[1]):
            
            seat = get_seat(pad, row+1, col+1)

            
            if fill_seat_rule(seat):
                
                update[row, col] = '#'
    
            elif vacate_seat_rule(seat):
                update[row, col] = 'L'
                
                
    return update

In [ ]:
first = data
second = next_turn(data)


while not np.array_equal(first, second):
    
    first, second = second, next_turn(second) 
    

In [ ]:
(first == '#').sum()

In [ ]:
(test == "#").sum() > 4

In [ ]:
test

In [ ]:
def get_raycast_seat(seats, row, col):
    
    
    

# Day 12

In [ ]:
raw_data = aoc.get_data(2020, 12)

In [ ]:
data = data.splitlines()

In [26]:
class Boat:
    
    def __init__(self):
        
        self.ew = 0
        self.ns = 0
        self.heading = 90
        
    
    def process_instructions(self, instructions):
        
        self.__init__()
        
        for instruction in instructions:
            
            self.process_instruction(instruction)
            
        print(abs(self.ew) + abs(self.ns))
        
    def process_instruction(self, instruction):
        
        preamble = instruction[0]
        
        action = int(instruction[1:])
        
        if preamble == 'F':
            
            heading = self.heading % 360
        
            if heading == 90:
                
                preamble = 'E'
                
            elif heading == 0:
                
                preamble = 'N'
                
            elif heading == 270:
                
                preamble = 'W'
                
            else:
                
                preamble = 'S'
        
        
        match preamble:
            
            case 'N':
                self.ns += action
                
            case 'S':
                self.ns -= action
                
            case 'E':
                self.ew += action
                
            case 'W':
                self.ew -= action
                
            case "R":
                self.heading += action
                
            case "L":
                self.heading -= action
                
                
                
    

In [27]:
boat = Boat()

In [28]:
boat.process_instructions(data)

845


In [30]:
np.array([1,0]) * 2

array([2, 0])

In [122]:
class Boat:
    
    def __init__(self):
        
        self.position = np.array([0,0], dtype=float)
        self.waypoint = np.array([10, 1], dtype=float)
        
    
    def process_instructions(self, instructions, debug=False):
        
        self.__init__()
        
        self.debug = debug
        
        for instruction in instructions:
            
            self.process_instruction(instruction, debug)
            
        print(np.abs(self.position).sum())
        
    def process_instruction(self, instruction, debug):
        
        preamble = instruction[0]
        
        action = int(instruction[1:])
        
        match preamble:
            
            case 'N':
                self.waypoint += np.array([0,1], dtype=float) * action
                
            case 'S':
                self.waypoint += np.array([0, -1], dtype=float) * action
                
            case 'E':
                self.waypoint += np.array([1,0],dtype=float) * action
                
            case 'W':
                self.waypoint += np.array([-1, 0],dtype=float) * action
                
            case "R":
                self.waypoint = self.rotate(action * -1.0, self.waypoint)
                
            case "L":
                self.waypoint = self.rotate(action, self.waypoint)
                
            case "F":
                self.position += self.waypoint * action
        
        if debug:        
            print(self.position, self.waypoint, instruction)
                
    def rotate(self, angle, vector):
        
        theta = np.radians(angle)
        c, s = np.cos(theta), np.sin(theta)
        R = np.array(((c, -s), (s, c)))
        
        return np.dot(R, vector)
        

In [123]:
boat = Boat()

In [124]:
boat.process_instructions(['F10', 'N3', 'F7', 'R90', 'F11'], True)

[100.  10.] [10.  1.] F10
[100.  10.] [10.  4.] N3
[170.  38.] [10.  4.] F7
[170.  38.] [  4. -10.] R90
[214. -72.] [  4. -10.] F11
286.0


In [125]:
boat.process_instructions(data)

27016.00000000003
